In [1]:
# To find ECD Control

import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')


In [1]:
import numpy as np
from qutip import *
from ECD_batch_optimizerV1 import BatchOptimizer
from Simulation_Classes_Single_ModeV4 import *
import matplotlib.pyplot as plt

C:\_Lib\python\Multimode-Conditional-Displacements\hpc_runs\single_mode_ecd\Simultaneous State Transfer\ECD_batch_optimizerV1.py:153: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (filename is not None and filename is not "")
C:\_Lib\python\Multimode-Conditional-Displacements\hpc_runs\single_mode_ecd\Simultaneous State Transfer\ECD_batch_optimizerV1.py:658: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  time_per_epoch = elapsed_time_s / epoch if epoch is not 0 else 0.0



Need tf version 2.3.0 or later. Using tensorflow version: 2.4.0



In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
df = None
columns=['time',  'layer', ]

In [4]:
def one_mode_fock_state(n): 
    return tensor(basis(2,0), basis(N, n))

In [5]:
N =15 # truncation

In [18]:
#The target oscillator state.

Fock = 1
psi_i = [one_mode_fock_state(5), one_mode_fock_state(0)] # initial state
psi_t = [one_mode_fock_state(0), one_mode_fock_state(5)] #target state

In [19]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 1, #circuit depth
'N_multistart' : 50, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 500, #number of epochs before termination
'epoch_size' : 10, #number of adam steps per epoch
'learning_rate' : 0.001, #adam learning rate
'term_fid' : 0.999, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : psi_i, #qubit tensor oscillator, start in |g> |0>
'target_states' : psi_t, #end in |e> |target>.
'name' : 'Fock %d' % Fock, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

#### Filename for angles to be stored

In [20]:
df = None 
fname = 'check5_jan30'

In [25]:
def main(layers, fname = fname, df = df):
    '''
    Vary layers, get fidelity
    '''
    filenum = 0
    for layer in layers: 
        #optimizer
        opt_params['N_blocks'] = layer
        opt = BatchOptimizer(**opt_params)
        opt.optimize()
        BO_fid = opt.best_fidelity()
        angles_filename = 'test_opt_angles_'+ str(filenum)+'.txt'
        opt.save_angles(filename = angles_filename)
        
        #pulses
        pulse_sim = ecd_pulse_single_mode(param_file = angles_filename,
                                 kappa = 1/(2e+6)) 
        pulse_sim.get_pulses()
        
        pulse_time = len(pulse_sim.alpha)
        
        df_new = pd.DataFrame([[ pulse_time, layer, BO_fid]], columns=columns)
        if df is None : 
            df = df_new
        else:
            df = df.append(df_new, ignore_index = True)
        df.to_csv(fname, index=False)
        filenum+=1

In [26]:
layers = [i for i in range(10, 20, 2)]

In [27]:
layers

[10, 12, 14, 16, 18]

In [28]:
main(layers)

None
Start time: 2023-01-30 12:49:53
Tensor("alphas:0", shape=(1, 50), dtype=complex64)
Tensor("alphas:0", shape=(10, 50), dtype=complex64)
 Epoch: 500 / 500 Max Fid: 0.885954 Avg Fid: 0.720074 Max dFid: 0.000341 Avg dFid: 0.000025 Elapsed time: 0:01:09.329107 Remaing time: 0:00:00.138680

Optimization stopped.  Reached maximum number of epochs. Terminal fidelity not reached.

optimization_type: state transfer
N_multistart: 50
N_blocks: 10
term_fid: 0.999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.001
epoch_size: 10
epochs: 500
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
user_angles: None
N_cav: 15
filename: Fock 1.h5

Best circuit parameters found:
betas:         [-0.72824+0.84795j  0.33775+0.26037j -2.035  +1.66231j -0.04652+1.33801j
 -0.50756-2.41386j -0.29812-0.93069j -1.98865-0.2045j  -1.31164+0.22334j
 -1.53517+0.54848j -0.71944+0.80837j]
alphas:        [0.+0.j]
phis (deg):    [   0.     

C:\Users\cssri\Anaconda3\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)


---------------------------
Final Displacement: 0.42645761343568517
Modified conditional displacement called
---------------------------
Final Displacement: 2.6276407625765397
Modified conditional displacement called
---------------------------
Final Displacement: 1.3388212365801797
Modified conditional displacement called
---------------------------
Final Displacement: 2.466647269246006
Modified conditional displacement called
---------------------------
Final Displacement: 0.9772712925256754
Modified conditional displacement called
---------------------------
Final Displacement: 1.9991346832181947
Modified conditional displacement called
---------------------------
Final Displacement: 1.3307938262175911
Modified conditional displacement called
---------------------------
Final Displacement: 1.6293113070465304
Modified conditional displacement called
---------------------------
Final Displacement: 1.0819910972780233


ValueError: 5 columns passed, passed data had 3 columns